<a href="https://colab.research.google.com/github/tvjoseph/Exercises/blob/master/JMJPFU_textSimilarity_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JMJPFU
16-July-2019

##### Text SImilarity experimentstion

IN this notebook we will experiment with various text similarity approaches. 
Lord bless this attempt of yours

Reference

https://github.com/tvjoseph/nlp-notebooks

https://github.com/nlptown/nlp-notebooks/blob/master/Simple%20Sentence%20Similarity.ipynb

In [0]:
import tensorflow_hub as hub
import tensorflow as tf

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")

In [0]:
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

In [0]:
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1))
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2))

In [0]:
sim_scores = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)

In [0]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    
    [gse_sims] = session.run(
            [sim_scores],
            feed_dict={
                sts_input1: ['a list of famous lines from chaucer  mainly from  the canterbury tales     geoffrey chaucer quotations from chaucer works of geoffrey chaucer the canterbury tales geoffrey chaucer chaucer  language arts   literature   british literature   medieval british literature   medieval british authors and their works   geoffrey chaucer   canterbury tales    grade 9 grade 10 grade 11 grade 12'],
                sts_input2: ['write routinely over extended time frames  time for research  reflection  and revision  and shorter time frames  a single sitting or a day or two  for a range of tasks  purposes  and audiences  common core state standards for ela   college and career anchor standards   writing   range of writing']
            })

In [7]:
gse_sims

array([0.33717754], dtype=float32)

In [0]:
embeddings = embed(["The Lord is my Shepherd"])

In [9]:
with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        print(sess.run(embeddings))

[[ 0.04326646 -0.01074986  0.00461051 -0.00794471 -0.0062413  -0.03105794
   0.09058522  0.02056552 -0.07636881  0.02500754 -0.03974541  0.0714503
   0.06028523  0.00282367  0.06272189  0.07630569 -0.04205645  0.00654582
   0.06048888 -0.04931925 -0.02174743  0.00104562 -0.04462577 -0.03347482
   0.01973441  0.04410719 -0.05066333  0.0213686   0.03138587 -0.01390363
  -0.06089529 -0.02689903 -0.04505111 -0.01514193 -0.04189098 -0.06267375
   0.02741842  0.0035394   0.07314964  0.1109685   0.00921521 -0.01026845
   0.01979977  0.02476061  0.08499485  0.01990691  0.02064779  0.02715402
   0.03147878 -0.05309079 -0.06373773  0.06833445  0.03256908 -0.10710588
   0.02942637 -0.0863785   0.01536449  0.05753248  0.05062891 -0.06959762
  -0.07518612 -0.03114081 -0.0242844  -0.00416078  0.00256969 -0.05644087
  -0.04570016  0.07918756 -0.02614533  0.04287481  0.08680259  0.02910409
   0.03940637  0.04713157  0.03563426 -0.0118231   0.04596676 -0.04013969
   0.02593838  0.02333431 -0.01930165  

In [0]:
embeddings.shape

TensorShape([Dimension(1), Dimension(512)])

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

with tf.Graph().as_default():
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
    embed = hub.Module(module_url)
    embeddings = embed(["A long sentence.", "single-word",
                      "http://example.com"])

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        print(sess.run(embeddings))

In [0]:
# Importing the library files

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

from gensim import corpora
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from pickle import dump
from pickle import load
from numpy.random import shuffle

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
documents = pd.read_csv("/content/drive/My Drive/ML_files/LanguageArts-GradeLevels.csv", encoding = "ISO-8859-1")
documents = documents[~(documents["Eval Description"].isna())]
documents = documents[~(documents["Eval Title"].isna())]
documents = documents[~(documents["Keywords"].isna())]
documents = documents[~(documents["Subject Node Lineage"].isna())]
documents = documents[~(documents["Grade Levels"].isna())]
print(len(documents))
documents.head()

46116


,Eval Code,Eval Title,Eval Description,Keywords,Grade Levels,Subject Node,Subject Node Lineage,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 377,Unnamed: 378,Unnamed: 379,Unnamed: 380,Unnamed: 381,Unnamed: 382,Unnamed: 383,Unnamed: 384,Unnamed: 385,Unnamed: 386,Unnamed: 387,Unnamed: 388,Unnamed: 389,Unnamed: 390,Unnamed: 391,Unnamed: 392,Unnamed: 393,Unnamed: 394,Unnamed: 395,Unnamed: 396,Unnamed: 397,Unnamed: 398,Unnamed: 399,Unnamed: 400,Unnamed: 401,Unnamed: 402,Unnamed: 403,Unnamed: 404,Unnamed: 405,Unnamed: 406,Unnamed: 407,Unnamed: 408,Unnamed: 409,Unnamed: 410,Unnamed: 411,Unnamed: 412,Unnamed: 413,Unnamed: 414,Unnamed: 415,Unnamed: 416
0,146,Barltett's Familiar Quotations: Geoffrey Chaucer,"A list of famous lines from Chaucer, mainly fr...","Geoffrey Chaucer|\The Canterbury Tales\""|Chau...",9|10|11|12,Canterbury Tales|Canterbury in England,Language Arts > Literature > British Literatu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,151,"HyperHistory Online: Roger Bacon, Philosopher","This is a very short, basic biography of Roger...",Roger Bacon,6|7|8|9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,152,University of St. Andrews: Roger Bacon,The University of St. Andrews publishes this b...,"\Opus Majus\""|Roger Bacon|University of St. An...",9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,154,University of St. Andrews: Roger Bacon (Image ...,"A black-and-white portrait of Roger Bacon, Eng...",Roger Bacon|University of St. Andrews,6|7|8|9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155,Catholic Encyclopedia (1907-14): Roger Bacon,Sophisticated biography of Bacon that discuses...,Roger Bacon|New Advent,9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Importing the training data with proper labels to combine the text with the previous data
documents2 = pd.read_csv("/content/drive/My Drive/ML_files/ELA Standards Resource Alignments with CASE Node Lineage.csv", encoding = "ISO-8859-1")
documents2 = documents2[~(documents2["Standard Description"].isna())]
documents2 = documents2[~(documents2["CASE Standard Lineage"].isna())]
print(len(documents2))
documents2.head()

12307


,Standard Node Code,Eval Code,Authority,Standard ID,Standard Title,Standard Description,Document Title,Subject,State Code,Alignment Status,CASE Standard Lineage
0,2410952459772773RU7,384329,NGA Center/CCSSO (CC),147514,CCSS.ELA-Literacy.W.11-12.10,Write routinely over extended time frames (tim...,Common Core State Standards,English Language Arts/Literacy - 2010,CC.LANG.10.,APPROVED,Common Core State Standards for ELA > College ...
1,2411435569600159uvL,67994256611135232i7skio,NGA Center/CCSSO (CC),200955,CCSS.ELA-Literacy.W.2.6,"With guidance and support from adults, use a v...",Common Core State Standards,English Language Arts/Literacy - 2010,CC.LANG.6.,APPROVED,Common Core State Standards for ELA > Grade 1 ...
2,2410952999677332qgw,313030,NGA Center/CCSSO (CC),147579,CCSS.ELA-Literacy.SL.11-12.6,Adapt speech to a variety of contexts and task...,Common Core State Standards,English Language Arts/Literacy - 2010,CC.LANG.6.,APPROVED,Common Core State Standards for ELA > Grade 11...
3,2411450824983128JM8,388664,NGA Center/CCSSO (CC),202570,CCSS.ELA-Literacy.W.7.10,Write routinely over extended time frames (tim...,Common Core State Standards,English Language Arts/Literacy - 2010,CC.LANG.10.,APPROVED,Common Core State Standards for ELA > Grade 11...
4,2411450925935932f3V,313030,NGA Center/CCSSO (CC),202579,CCSS.ELA-Literacy.SL.7.4,"Present claims and findings, emphasizing salie...",Common Core State Standards,English Language Arts/Literacy - 2010,CC.LANG.4.,APPROVED,Common Core State Standards for ELA > Grade 7 ...


In [0]:
for i in range(0,len(documents)):
    documents['Unnamed: 7'].iloc[i] = ' '.join(list(map(lambda x : 'Grade' + ' ' + x ,documents.iloc[i]['Grade Levels'].split('|'))))


In [22]:
documents = documents.rename(index=str,columns={'Unnamed: 7':'Grade'})
documents

,Eval Code,Eval Title,Eval Description,Keywords,Grade Levels,Subject Node,Subject Node Lineage,Grade,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 377,Unnamed: 378,Unnamed: 379,Unnamed: 380,Unnamed: 381,Unnamed: 382,Unnamed: 383,Unnamed: 384,Unnamed: 385,Unnamed: 386,Unnamed: 387,Unnamed: 388,Unnamed: 389,Unnamed: 390,Unnamed: 391,Unnamed: 392,Unnamed: 393,Unnamed: 394,Unnamed: 395,Unnamed: 396,Unnamed: 397,Unnamed: 398,Unnamed: 399,Unnamed: 400,Unnamed: 401,Unnamed: 402,Unnamed: 403,Unnamed: 404,Unnamed: 405,Unnamed: 406,Unnamed: 407,Unnamed: 408,Unnamed: 409,Unnamed: 410,Unnamed: 411,Unnamed: 412,Unnamed: 413,Unnamed: 414,Unnamed: 415,Unnamed: 416
0,146,Barltett's Familiar Quotations: Geoffrey Chaucer,"A list of famous lines from Chaucer, mainly fr...","Geoffrey Chaucer|\The Canterbury Tales\""|Chau...",9|10|11|12,Canterbury Tales|Canterbury in England,Language Arts > Literature > British Literatu...,Grade 9 Grade 10 Grade 11 Grade 12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,151,"HyperHistory Online: Roger Bacon, Philosopher","This is a very short, basic biography of Roger...",Roger Bacon,6|7|8|9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 6 Grade 7 Grade 8 Grade 9 Grade 10 Grade...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,152,University of St. Andrews: Roger Bacon,The University of St. Andrews publishes this b...,"\Opus Majus\""|Roger Bacon|University of St. An...",9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 9 Grade 10 Grade 11 Grade 12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,154,University of St. Andrews: Roger Bacon (Image ...,"A black-and-white portrait of Roger Bacon, Eng...",Roger Bacon|University of St. Andrews,6|7|8|9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 6 Grade 7 Grade 8 Grade 9 Grade 10 Grade...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155,Catholic Encyclopedia (1907-14): Roger Bacon,Sophisticated biography of Bacon that discuses...,Roger Bacon|New Advent,9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 9 Grade 10 Grade 11 Grade 12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,171,University of Virginia: The Morte de Malory [T...,"Describes the influence of \Morte d'Arthur\"" u...","\A Connecticut Yankee in King Arthur's Court\""...",9|10|11|12,Morte dArthur,Language Arts > Literature > British Literatu..

In [0]:
# Let us now try to clean up the Keywords
for i in range(0,len(documents)):
    docsplit = set(documents['Keywords'].iloc[i].split('|'))
    mylist = []
    for d in docsplit:
        mylist.append(d)
    seclist = ' '.join(mylist)
    newlist = seclist.split('\\')
    seclist = ' '.join(newlist)
    documents['Unnamed: 8'].iloc[i] = [re.sub("[^a-zA-Z]", " ", seclist)][0]
    print(i)

In [24]:
documents = documents.rename(index=str,columns={'Unnamed: 8':'Clean Keywords'})
documents.head()

,Eval Code,Eval Title,Eval Description,Keywords,Grade Levels,Subject Node,Subject Node Lineage,Grade,Clean Keywords,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 377,Unnamed: 378,Unnamed: 379,Unnamed: 380,Unnamed: 381,Unnamed: 382,Unnamed: 383,Unnamed: 384,Unnamed: 385,Unnamed: 386,Unnamed: 387,Unnamed: 388,Unnamed: 389,Unnamed: 390,Unnamed: 391,Unnamed: 392,Unnamed: 393,Unnamed: 394,Unnamed: 395,Unnamed: 396,Unnamed: 397,Unnamed: 398,Unnamed: 399,Unnamed: 400,Unnamed: 401,Unnamed: 402,Unnamed: 403,Unnamed: 404,Unnamed: 405,Unnamed: 406,Unnamed: 407,Unnamed: 408,Unnamed: 409,Unnamed: 410,Unnamed: 411,Unnamed: 412,Unnamed: 413,Unnamed: 414,Unnamed: 415,Unnamed: 416
0,146,Barltett's Familiar Quotations: Geoffrey Chaucer,"A list of famous lines from Chaucer, mainly fr...","Geoffrey Chaucer|\The Canterbury Tales\""|Chau...",9|10|11|12,Canterbury Tales|Canterbury in England,Language Arts > Literature > British Literatu...,Grade 9 Grade 10 Grade 11 Grade 12,works of Geoffrey Chaucer Geoffrey Chaucer Ch...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,151,"HyperHistory Online: Roger Bacon, Philosopher","This is a very short, basic biography of Roger...",Roger Bacon,6|7|8|9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 6 Grade 7 Grade 8 Grade 9 Grade 10 Grade...,Roger Bacon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,152,University of St. Andrews: Roger Bacon,The University of St. Andrews publishes this b...,"\Opus Majus\""|Roger Bacon|University of St. An...",9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 9 Grade 10 Grade 11 Grade 12,Roger Bacon Opus Majus University of St An...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,154,University of St. Andrews: Roger Bacon (Image ...,"A black-and-white portrait of Roger Bacon, Eng...",Roger Bacon|University of St. Andrews,6|7|8|9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 6 Grade 7 Grade 8 Grade 9 Grade 10 Grade...,Roger Bacon University of St Andrews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155,Catholic Encyclopedia (1907-14): Roger Bacon,Sophisticated biography of Bacon that discuses...,Roger Bacon|New Advent,9|10|11|12,Roger Bacon,Language Arts > Literature > British Literatu...,Grade 9 Grade 10 Grade 11 Grade 12,Roger Bacon New Advent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
import nltk
nltk.download('stopwords')  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def doc_to_words( raw_data ):
    #letters_only = re.sub("[^a-zA-Z]", " ", raw_data) 
    words = raw_data.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join(words ))

def doc_to_words2( raw_data ):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_data) 
    words = letters_only.lower().split('\\')                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join(words ))

def doc_to_words3( raw_data ):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_data) 
    words = letters_only.lower().split('|')                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join(words ))
def doc_to_words4( raw_data ):
    letters_only = re.sub("[^a-zA-Z]", " ", raw_data) 
    words = letters_only.lower().split('>')                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join(words ))
def doc_to_words5( raw_data ):
    #letters_only = re.sub("[^a-zA-Z]", " ", raw_data) 
    words = raw_data.lower().split('>')                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join(words ))

In [28]:
resdata1 = documents["Eval Description"]
resdata2 = documents["Subject Node"]
resdata3 = documents["Clean Keywords"]
resdata4 = documents["Subject Node Lineage"]
resdata5 = documents["Grade"]

clean_train_resdata1 = []
clean_train_resdata2 = []
clean_train_resdata3 = []
clean_train_resdata4 = []
clean_train_resdata5 = []

for i in range( 0, len(resdata1) ):
    clean_train_resdata1.append( doc_to_words2( resdata1.iloc[i] ) )
    clean_train_resdata2.append( doc_to_words3( resdata2.iloc[i] ) )
    clean_train_resdata3.append( doc_to_words( resdata3.iloc[i] ) )
    clean_train_resdata4.append( doc_to_words4( resdata4.iloc[i] ) )
    clean_train_resdata5.append( doc_to_words( resdata5.iloc[i] ) )
    
print(len(clean_train_resdata1))    
print(len(clean_train_resdata2))
print(len(clean_train_resdata3))
print(len(clean_train_resdata4))
print(len(clean_train_resdata5))


# Combining both these data sets into one
combResdata = list(zip(clean_train_resdata1,clean_train_resdata3,clean_train_resdata4, clean_train_resdata5)) # ,clean_train_resdata3, clean_train_resdata2
print(len(combResdata))
combResdata

# Combinig both the strings for resources
Resdata = []
for i, _d in enumerate(combResdata):
    Resdata.append(_d[0] +' ' + _d[1]+' ' + _d[2]+' ' + _d[3]) #  

46116
46116
46116
46116
46116
46116


In [29]:
stddata1 = documents2["Standard Description"]
stddata2 = documents2["CASE Standard Lineage"]

clean_train_stddata1 = []
clean_train_stddata2 = []

for i in range( 0, len(stddata1) ):
    clean_train_stddata1.append( doc_to_words2( stddata1.iloc[i] ) )
    clean_train_stddata2.append( doc_to_words5( stddata2.iloc[i] ) )
print(len(clean_train_stddata1))    
print(len(clean_train_stddata2))


# Combining both these data sets into one
combStddata = list(zip(clean_train_stddata1, clean_train_stddata2))
print(len(combStddata))


# Combinig both the strings for resources
Stddata = []
for i, _d in enumerate(combStddata):
    Stddata.append(_d[0] +' ' + _d[1]) 
    
print(len(Stddata))

12307
12307
12307
12307


In [0]:
d1 = Stddata[0]
d2 = Resdata[0]

In [0]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    
    [gse_sims] = session.run(
            [sim_scores],
            feed_dict={
                sts_input1: [d1],
                sts_input2: [d2]
            })

In [0]:
simMattf = np.zeros((10,10))
for i in range(len(Stddata)):
    print(i)
    d2 = Stddata[i]
    for j in range(len(Resdata)):
        d1 = Resdata[j]
        with tf.Session() as session:
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
            [gse_sims] = session.run(
            [sim_scores],
            feed_dict={
                sts_input1: [d1],
                sts_input2: [d2]
            })
            simMattf[i,j] = gse_sims[0]  
        
        
        
        
        
            

0
